# Sharpe Ratio

This notebook has been created for active management of the SP500 ETF. It allowed to maximize the sharpe ratio of all the equities include in the SP500.

In [6]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize

#Step 1: Fetch the S&P 500 tickers
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
table = pd.read_html(url)
sp500_tickers = table[0]['Symbol'].tolist()

#Fix ticker symbols for Yahoo Finance
sp500_tickers = [ticker.replace('.', '-') for ticker in sp500_tickers]

#Step 2: Download adjusted closing prices
data = yf.download(sp500_tickers, start="2020-01-01", end="2025-01-01")['Close']

#Step 3: Clean the data (drop tickers with too many missing values)
data = data.dropna(axis=1, thresh=int(len(data) * 0.7))  # Keep only stocks with 70%+ data available

#Step 4: Compute daily returns and filter invalid stocks
returns = data.pct_change().dropna()

#Step 5: Compute annualized return & covariance matrix
mean_returns = returns.mean() * 252
cov_matrix = returns.cov() * 252

#Step 6: Validate the covariance matrix
if cov_matrix.isnull().values.any():
    print("Covariance matrix contains NaN values. Fixing it...")
    cov_matrix = cov_matrix.fillna(0)

#Step 7: Ensure valid data before optimization
if np.all(mean_returns == 0):
    raise ValueError("Mean returns are all zeros. Data issue?")
if np.all(cov_matrix == 0):
    raise ValueError("Covariance matrix is all zeros. Cannot optimize.")

#Step 8: Select the top 100 stocks with the highest mean return
top_stocks = mean_returns.nlargest(100).index
returns = returns[top_stocks]
mean_returns = mean_returns[top_stocks]
cov_matrix = cov_matrix.loc[top_stocks, top_stocks]

#Step 9: Set up the optimization parameters
risk_free_rate = 0.04  # 4% risk-free rate

#Define the negative Sharpe ratio function
def negative_sharpe(weights, mean_returns, cov_matrix, risk_free_rate):
    portfolio_return = np.dot(weights, mean_returns)
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std
    return -sharpe_ratio  # Negative since we minimize

#Constraints: Weights sum to 1
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

#Bounds: No short-selling (0% to 100%)
bounds = tuple((0, 1) for _ in range(len(top_stocks)))

#Step 10: Use an improved initial weight guess (inverse volatility)
inv_vol = 1 / returns.std()
initial_weights = inv_vol / np.sum(inv_vol)

#Step 11: Run the optimization
optimized_result = minimize(
    negative_sharpe,
    initial_weights,
    args=(mean_returns, cov_matrix, risk_free_rate),
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

#Step 12: Extract and display optimal weights
optimal_weights = optimized_result.x
print("\n📊 **Optimized Portfolio Weights:**")
for ticker, weight in sorted(zip(top_stocks, optimal_weights), key=lambda x: x[1], reverse=True)[:10]:
    print(f"{ticker}: {weight:.2%}")

# Step 13: Compute and display final portfolio statistics
optimal_return = np.dot(optimal_weights, mean_returns)
optimal_volatility = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights)))
optimal_sharpe = (optimal_return - risk_free_rate) / optimal_volatility

print(f"\n Expected Annual Return: {optimal_return:.2%}")
print(f" Expected Volatility: {optimal_volatility:.2%}")
print(f" Optimized Sharpe Ratio: {optimal_sharpe:.2f}")


[*********************100%***********************]  503 of 503 completed



📊 **Optimized Portfolio Weights:**
MCK: 18.12%
LLY: 15.85%
TRGP: 12.61%
VST: 8.64%
CBOE: 8.26%
IRM: 7.89%
PGR: 5.18%
ORLY: 3.90%
AVGO: 3.72%
AXON: 3.60%

 Expected Annual Return: 43.30%
 Expected Volatility: 16.42%
 Optimized Sharpe Ratio: 2.39
